In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Caminho do dataset
dataset_path = "../datasets/XWines_Full_100K_wines.csv"
df = pd.read_csv(dataset_path)

# Função para visualizar os dados iniciais
def visualize_initial_data(df):
    """
    Exibe uma pré-visualização dos primeiros registros do dataset e informações gerais sobre o mesmo.
    """
    print("\n# Visão Geral dos Dados Iniciais #")
    print(f"\n{df.head()}")
    print("\n# Informações sobre o Dataset #")
    print(df.info())

visualize_initial_data(df)

# Seleção de colunas relevantes
df = df[["Grapes", "Harmonize"]]

# Mapeamento das categorias de prato
dish_mapping = {
    'Beef': 'Meat', 'Lamb': 'Meat', 'Pork': 'Meat', 'Veal': 'Meat', 'Game Meat': 'Meat',
    'Duck': 'Meat', 'Ham': 'Meat', 'Cold Cuts': 'Meat', 'Cured Meat': 'Meat',
    'Poultry': 'Poultry', 'Chicken': 'Poultry',
    'Rich Fish': 'Fish & Seafood', 'Lean Fish': 'Fish & Seafood', 'Shellfish': 'Fish & Seafood',
    'Seafood': 'Fish & Seafood', 'Sushi': 'Fish & Seafood', 'Sashimi': 'Fish & Seafood',
    'Codfish': 'Fish & Seafood', 'Fish': 'Fish & Seafood', 'Grilled': 'Fish & Seafood',
    'Cheese': 'Cheese', 'Soft Cheese': 'Cheese', 'Hard Cheese': 'Cheese', 'Blue Cheese': 'Cheese',
    'Matured Cheese': 'Cheese', 'Goat Cheese': 'Cheese', 'Mild Cheese': 'Cheese',
    'Pasta': 'Pasta', 'Tagliatelle': 'Pasta', 'Lasagna': 'Pasta', 'Paella': 'Fish & Seafood', 'Pizza': 'Pasta',
    'Vegetarian': 'Vegetarian & Vegan', 'Mushrooms': 'Vegetarian & Vegan', 'Salad': 'Vegetarian & Vegan',
    'Fruit': 'Vegetarian & Vegan', 'Tomato Dishes': 'Vegetarian & Vegan', 'Beans': 'Vegetarian & Vegan',
    'Eggplant Parmigiana': 'Vegetarian & Vegan', 'Light Stews': 'Vegetarian & Vegan',
    'Appetizer': 'Appetizers & Snacks', 'Snack': 'Appetizers & Snacks', 'Aperitif': 'Appetizers & Snacks',
    'Sweet Dessert': 'Desserts', 'Cake': 'Desserts', 'Chocolate': 'Desserts', 'Cookies': 'Desserts',
    'Spicy Food': 'Spicy Food', 'Curry Chicken': 'Spicy Food', 'Asian Food': 'Spicy Food',
    'Barbecue': 'Meat', 'Roast': 'Meat'
}

# Função para processar a coluna "Harmonize"
def preprocess_harmonize_column(df):
    """
    Processa e mapeia a coluna 'Harmonize', convertendo as categorias de pratos para uma categoria geral.
    """
    df = df.copy()
    df["Harmonize"] = df["Harmonize"].apply(lambda x: eval(x) if isinstance(x, str) else x)
    df = df.explode("Harmonize")
    df["Dish Category"] = df["Harmonize"].map(dish_mapping)
    df = df.dropna(subset=["Dish Category"])
    df = df.drop(columns="Harmonize")
    return df

df = preprocess_harmonize_column(df)

# Função para visualizar os dados processados
def visualize_processed_data(df):
    """
    Exibe uma pré-visualização dos dados após o processamento, com as categorias de pratos.
    """
    print("\n# Dados Processados #")
    print(f"\n{df.head()}")
    print("\n# Categorias de Prato Únicas #")
    print(df["Dish Category"].unique())

visualize_processed_data(df)

# Codificação one-hot para as categorias de pratos
df = pd.get_dummies(df, columns=["Dish Category"], prefix="Dish")

# Processamento da coluna "Grapes"
def preprocess_grapes_column(df):
    """
    Processa a coluna 'Grapes', mantendo as 20 uvas mais frequentes e agrupando as demais como 'Other'.
    """
    df = df.copy()
    df["Grapes"] = df["Grapes"].apply(lambda x: x.split(',') if isinstance(x, str) else x)
    top_grapes = df["Grapes"].explode().value_counts().head(20).index
    df["Grapes"] = df["Grapes"].apply(lambda x: [g if g in top_grapes else 'Other' for g in x])
    df = df.explode("Grapes")
    df = pd.get_dummies(df, columns=["Grapes"], prefix="Grape")
    return df

df = preprocess_grapes_column(df)

# Combina as características one-hot
feature_columns = [col for col in df.columns if col.startswith("Dish_") or col.startswith("Grape_")]
X = df[feature_columns]
y = df[[col for col in df.columns if col.startswith("Grape_")]]

# Redução de dimensionalidade com PCA
pca = PCA(n_components=10)  # Reduzi para 10 componentes principais
X_pca = pca.fit_transform(X)

# Divisão entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Treinamento do classificador Random Forest multi-output
clf = MultiOutputClassifier(RandomForestClassifier(random_state=42, n_estimators=50, n_jobs=-1))  # Reduzi o número de árvores para 50
clf.fit(X_train, y_train)

# Predições
y_pred = clf.predict(X_test)

# Função para avaliar o modelo
def evaluate_model(y_test, y_pred):
    """
    Exibe o relatório de classificação do modelo treinado.
    """
    print("\n# Relatório de Classificação #")
    print(classification_report(y_test, y_pred, target_names=y.columns))

evaluate_model(y_test, y_pred)

# Importância das características (PCA)
print("\n# Razão de Variância Explicada pelo PCA #")
print(pca.explained_variance_ratio_)

# Função para testar categorias de pratos específicas
def test_dish_category(clf, category, pca, X):
    """
    Testa a recomendação de variedades de uvas para uma categoria específica de prato.
    """
    # Inicializa o vetor de características com zeros
    test_vector = np.zeros(X.shape[1])

    # Encontre o índice correspondente à categoria de prato no DataFrame
    dish_column = f"Dish_{category}"

    # Verifique se a categoria de prato está na lista de colunas
    if dish_column in X.columns:
        # Marque a posição correta no vetor de características para a categoria
        test_vector[X.columns.get_loc(dish_column)] = 1
    else:
        print(f"A categoria '{category}' não está presente nas colunas de características.")
        return
    
    # Aplique a transformação PCA no vetor de teste
    test_vector_pca = pca.transform([test_vector])
    
    # Faça a predição com o classificador
    prediction = clf.predict(test_vector_pca)[0]

    # Obtenha as uvas recomendadas
    predicted_grapes = [y.columns[i] for i, val in enumerate(prediction) if val == 1]
    
    print(f"\n# Recomendação de Uvas para a Categoria '{category}' #")
    if predicted_grapes:
        print(f"Variedades de uvas recomendadas: {predicted_grapes}")
    else:
        print("Nenhuma variedade de uva recomendada para esta categoria.")

# Teste de exemplo
test_dish_category(clf, 'Meat', pca, X)


# Visão Geral dos Dados Iniciais #

   WineID                         WineName       Type  \
0  100001               Espumante Moscatel  Sparkling   
1  100002                       Ancellotta        Red   
2  100003               Cabernet Sauvignon        Red   
3  100004                   Virtus Moscato      White   
4  100005  Maison de Ville Cabernet-Merlot        Red   

                       Elaborate                            Grapes  \
0                  Varietal/100%                ['Muscat/Moscato']   
1                  Varietal/100%                    ['Ancellotta']   
2                  Varietal/100%            ['Cabernet Sauvignon']   
3                  Varietal/100%                ['Muscat/Moscato']   
4  Assemblage/Bordeaux Red Blend  ['Cabernet Sauvignon', 'Merlot']   

                                           Harmonize   ABV           Body  \
0                 ['Pork', 'Rich Fish', 'Shellfish']   7.5  Medium-bodied   
1  ['Beef', 'Barbecue', 'Codfish', 'Pasta', '

C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Sistrade\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(



# Recomendação de Uvas para a Categoria 'Meat' #
Nenhuma variedade de uva recomendada para esta categoria.


# Resultados Obtidos

## Visão Geral dos Dados

O dataset contém informações sobre vinhos, incluindo detalhes como o nome, tipo, uvas utilizadas, e pratos que harmonizam com cada vinho. Após a análise inicial, a tabela foi reduzida para as colunas "Grapes" e "Harmonize", que representam, respectivamente, as uvas e os pratos recomendados para harmonização.

## Processamento de Dados

- A coluna "Harmonize" foi mapeada para categorias gerais de pratos, como "Meat", "Fish & Seafood", "Cheese", entre outras. Este mapeamento foi realizado com base nas categorias fornecidas no dataset e aplicado corretamente aos dados.
  
- A coluna "Grapes" foi processada mantendo as 20 uvas mais frequentes, agrupando as demais como 'Other'. Isso permitiu uma base de dados mais compacta e adequada para análise e classificação.

## Codificação One-Hot

Foi aplicada a codificação one-hot para as colunas de categorias de pratos ("Harmonize") e tipos de uvas ("Grapes"). Essa técnica transforma variáveis categóricas em variáveis binárias, facilitando a utilização dessas variáveis em modelos de Machine Learning.

## PCA (Análise de Componentes Principais)

A técnica de PCA foi utilizada para redução de dimensionalidade, com a retenção de 10 componentes principais. A razão de variância explicada pelos componentes principais foi a seguinte:

- **Razão de Variância Explicada**: [0.22758457, 0.18236598, 0.09441158, 0.0610507, 0.04207341, 0.03791263, 0.03432576, 0.03362562, 0.0281702, 0.02632314]

Esses componentes principais explicam uma significativa parte da variabilidade dos dados, permitindo reduzir a complexidade enquanto preserva informações importantes.

## Treinamento do Modelo

Foi utilizado um classificador Random Forest com múltiplas saídas (`MultiOutputClassifier`), para prever as uvas recomendadas para diferentes categorias de pratos. O modelo foi treinado com uma divisão de 80% dos dados para treinamento e 20% para teste.

## Desempenho do Modelo

A avaliação do modelo resultou em um relatório de classificação. No entanto, surgiram desafios, como a presença de categorias de pratos para as quais o modelo não conseguiu prever uvas adequadas. Isso gerou uma **precisão indefinida** para algumas categorias, onde nenhuma uva foi prevista. Este resultado indica possíveis problemas com a distribuição ou diversidade dos dados.

## Recomendação de Uvas

Foi realizada uma recomendação de uvas para a categoria "Meat". No entanto, o modelo não conseguiu identificar nenhuma uva recomendada para esta categoria. Isso pode ser devido a limitações do modelo em identificar padrões adequados ou à necessidade de ajustes adicionais no treinamento ou nos dados.

## Conclusões e Próximos Passos

- O modelo de Random Forest mostrou potencial, mas ajustes na modelagem e no pré-processamento dos dados podem ser necessários para melhorar as recomendações.
- A análise de componentes principais (PCA) contribuiu para a redução da dimensionalidade, mantendo a variabilidade dos dados.
- Será necessário ajustar as categorias de pratos ou a abordagem de treinamento para garantir que o modelo possa gerar recomendações de uvas mais precisas.

Esses resultados sugerem que o modelo pode ser melhorado com a adição de mais dados ou ajustes na forma como as categorias de pratos e uvas são tratadas.